In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e1/sample_submission.csv
/kaggle/input/playground-series-s5e1/train.csv
/kaggle/input/playground-series-s5e1/test.csv


In [2]:
df = pd.read_csv("/kaggle/input/playground-series-s5e1/train.csv")

In [3]:
df['date'].unique()

array(['2010-01-01', '2010-01-02', '2010-01-03', ..., '2016-12-29',
       '2016-12-30', '2016-12-31'], dtype=object)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230130 entries, 0 to 230129
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   id        230130 non-null  int64  
 1   date      230130 non-null  object 
 2   country   230130 non-null  object 
 3   store     230130 non-null  object 
 4   product   230130 non-null  object 
 5   num_sold  221259 non-null  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 10.5+ MB


In [5]:
df

,id,date,country,store,product,num_sold
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN
1,1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0
2,2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0
3,3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0
4,4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0
...,...,...,...,...,...,...
230125,230125,2016-12-31,Singapore,Premium Sticker Mart,Holographic Goose,466.0
230126,230126,2016-12-31,Singapore,Premium Sticker Mart,Kaggle,2907.0
230127,230127,2016-12-31,Singapore,Premium Sticker Mart,Kaggle Tiers,2299.0
230128,230128,2016-12-31,Singapore,Premium Sticker Mart,Kerneler,1242.0


In [6]:
df.columns

Index(['id', 'date', 'country', 'store', 'product', 'num_sold'], dtype='object')

In [7]:
df['num_sold'] = df['num_sold'].fillna(df['num_sold'].mean())


In [8]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
label_encoders = {}
for col in ['country', 'store', 'product']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

In [9]:
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df = df.drop(['id', 'date'], axis=1)

In [10]:
scaler = StandardScaler()
X = scaler.fit_transform(df.drop('num_sold', axis=1))
y = df['num_sold'].values

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim

In [13]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

In [14]:
class SalesPredictor(nn.Module):
    def __init__(self, input_dim):
        super(SalesPredictor, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x


In [15]:
input_dim = X_train.shape[1]
model = SalesPredictor(input_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 1000
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    predictions = model(X_train_tensor)
    loss = criterion(predictions, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

Epoch 10/1000, Loss: 1022324.1875
Epoch 20/1000, Loss: 1020703.3125
Epoch 30/1000, Loss: 1018327.2500
Epoch 40/1000, Loss: 1014961.7500
Epoch 50/1000, Loss: 1010307.0625
Epoch 60/1000, Loss: 1004015.1875
Epoch 70/1000, Loss: 995738.6250
Epoch 80/1000, Loss: 985131.2500
Epoch 90/1000, Loss: 971864.0000
Epoch 100/1000, Loss: 955642.9375
Epoch 110/1000, Loss: 936205.6250
Epoch 120/1000, Loss: 913332.0625
Epoch 130/1000, Loss: 886960.8125
Epoch 140/1000, Loss: 857182.5625
Epoch 150/1000, Loss: 824255.8750
Epoch 160/1000, Loss: 788621.6875
Epoch 170/1000, Loss: 750909.1250
Epoch 180/1000, Loss: 711929.5625
Epoch 190/1000, Loss: 672652.8125
Epoch 200/1000, Loss: 634158.8750
Epoch 210/1000, Loss: 597566.8125
Epoch 220/1000, Loss: 563934.0000
Epoch 230/1000, Loss: 534141.9375
Epoch 240/1000, Loss: 508788.8438
Epoch 250/1000, Loss: 488115.3750
Epoch 260/1000, Loss: 471991.0625
Epoch 270/1000, Loss: 459958.3438
Epoch 280/1000, Loss: 451330.5938
Epoch 290/1000, Loss: 445320.9375
Epoch 300/1000, L

In [16]:
model.eval()
with torch.no_grad():
    test_predictions = model(X_test_tensor)
    test_loss = criterion(test_predictions, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')


Test Loss: 374349.4062
